In [24]:
pip install requests langchain_core

Note: you may need to restart the kernel to use updated packages.


In [25]:
from typing import TypedDict
from langchain_core.prompts import PromptTemplate
import requests
import json

class Parameter(TypedDict):
    type: str
    description: str
    enum: list = None

class Function(TypedDict):
    name: str
    description: str
    parameters: dict[Parameter]

class Tool(TypedDict):
    type: str
    function: Function

class LLM_Prompt(TypedDict):
    tools: list[Tool]
    prompt: str
    model: str = "mistral:latest"

ollama_url = "http://127.0.0.1:11434"


def promptTemplate(tools: list[Tool], prompt: str):
    tools = json.dumps(tools)
    prompt = PromptTemplate.from_template("""[AVAILABLE_TOOLS]{tools}[/AVAILABLE_TOOLS][INST] {prompt} [/INST]""").format(tools=tools, prompt=prompt)
    return prompt

def promptLLM(prompt: LLM_Prompt):
    data = {
        "model":  prompt["model"],
        "prompt": promptTemplate(prompt["tools"], prompt["prompt"]),
        "stream": False,
        "raw": True
    }
    response = requests.post(ollama_url + "/api/generate", json=data)
    llm_res = json.loads(response.text)["response"]
    if "[TOOL_CALLS]" in llm_res:
        try:
            tool_calls = json.loads(llm_res.split("[TOOL_CALLS]")[1].split("\n")[0])
        except:
            tool_calls = llm_res.split("[TOOL_CALLS]")[1]
        return tool_calls
        
    return llm_res

In [26]:
response = promptLLM(LLM_Prompt(
    prompt="What is the weather in San Francisco?",
    tools=[{
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city, and state, e.g. Berlin, Germany. If the location is in the US, include the state, e.g. Seattle, Washington, USA."
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location."
                    }
                },
                "required": ["location", "unit"]
            }
        }
    }],
    model="mistral:latest"
))

response

[{'name': 'get_current_weather',
  'arguments': {'location': 'San Francisco, California, USA',
   'unit': 'fahrenheit'}}]

In [27]:
response = promptLLM(LLM_Prompt(
    prompt="What is the weather in Colmar?",
    tools=[{
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city, and state, e.g. Berlin, Germany. If the location is in the US, include the state, e.g. Seattle, Washington, USA."
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location."
                    }
                },
                "required": ["location", "unit"]
            }
        }
    }],
    model="mistral:latest"
))

response

[{'name': 'get_current_weather',
  'arguments': {'location': 'Colmar, France', 'unit': 'celsius'}}]

In [28]:
response = promptLLM(LLM_Prompt(
    prompt="How much is 5 shares of AAPL?",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_stock_price",
                "description": "Get the latest stock price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {
                            "type": "string",
                            "description": "The stock ticker symbol, e.g. AAPL",
                        },
                        "shares": {
                            "type": "int",
                            "description": "The number of shares to get the price for. Default is 1.",
                        }
                    },
                    "required": ["ticker"],
                },
            }
        }
    ],
    model="mistral:latest"
))

response

[{'name': 'get_stock_price', 'arguments': {'ticker': 'AAPL', 'shares': 5}}]

In [29]:
response = promptLLM(LLM_Prompt(
    prompt="How much is 5 shares of AAPL?",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_stock_price",
                "description": "Get the latest stock price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {
                            "type": "string",
                            "description": "The stock ticker symbol, e.g. AAPL",
                        }
                    },
                    "required": ["ticker"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "calculate_share_price",
                "description": "Calculate the cost of a given number of shares",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "number_of_shares": {
                            "type": "int",
                            "description": "The number of shares to buy",
                        },
                        "price": {
                            "type": "float",
                            "description": "The price per share",
                        },
                    },
                    "required": ["number_of_shares", "price"],
                },
            }
        }
    ],
    model="mistral:latest"
))
response

' To calculate the cost of 5 shares of AAPL, I\'ll first need to find out the current price of AAPL. Here\'s how we can do it:\n\n```javascript\nconst stockPrice = get_stock_price({"ticker": "AAPL"});\n```\n\nNow that I have the price per share, let me calculate the cost for 5 shares:\n\n```javascript\nconst numberOfShares = 5;\nconst pricePerShare = stockPrice;\nconst totalCost = calculate_share_price({"number_of_shares": numberOfShares, "price": pricePerShare});\n\nconsole.log(`The cost of ${numberOfShares} shares of AAPL is: $${totalCost}`);\n```'